## Casual Analysis of Risk factors associated with depression

In [5]:
import numpy as np
import pandas as pd

import os
from functools import reduce  

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import chi2, f_regression
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import random
from prettytable import PrettyTable

## Load Data

In [6]:
RANDOM_SEED = 4200

# full data
full_data = pd.DataFrame()

# from each file
for filename in os.listdir('data'):
    if(filename.endswith(".XPT")):
        file = os.path.join('data', filename)
        newcols = pd.read_sas(file, encoding='UTF-8')
        
        if(len(full_data) == 0):
            full_data = newcols
        else:
            full_data = pd.merge(full_data, newcols, on = "SEQN", how = "outer")

#in case we want the file in csv
# full_data.to_csv('data/data_combined.csv')

print(full_data.shape)

(9254, 310)


In [7]:
cols = full_data.loc[:,"DPQ010":"DPQ090"].columns
valid = full_data.copy()
valid[cols] = valid[cols].round()

# only care about values 0-3
for i in range(len(cols)):
    valid = valid.loc[valid[cols[i]] <=3]

valid["sum_DPQ"] = valid[cols].sum(axis="columns", skipna = True)
print(valid.shape)

(5068, 311)


In [8]:
valid.head()

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,WHD080L,WHQ225,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200,sum_DPQ
0,93705.0,1.0,7.000000e+00,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.000000e+00,...,NaN,4.0,150.0,130.0,63.0,170.0,62.0,2.0,NaN,0.0
1,93706.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,150.0,17.0,2.0,NaN,0.0
2,93708.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,118.0,100.0,60.0,120.0,40.0,2.0,NaN,0.0
4,93711.0,1.0,5.000000e+00,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,5.0,142.0,148.0,69.0,152.0,33.0,2.0,NaN,2.0
5,93712.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,5.0,NaN,NaN,NaN,137.0,17.0,2.0,NaN,1.0


In [9]:
# chosen variables
features = ['SEQN','RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDHHSIZ',
            'INDHHIN2','INDFMIN2', 'BMXBMI', 'URXUAS', 'URXUHG', 'ALQ130', 'DUQ217', 'FSD230', 'INDFMMPI',
            'PAD680', 'OCQ180', 'OCQ210', 'SMQ040', 'SMD650', 'SLD012', 'FIAINTRP', 'FIAPROXY', 'FSD032A', 
            'FSD032B', 'FSD032C', 'FSD061', 'HUQ051', 'HUQ090', 'PAD615','SLQ050','OCQ380', 'PAD630', 'PAD660', 'SLQ120','sum_DPQ']

subdata = full_data.filter(items = features)
subdata = subdata[subdata.RIDAGEYR >= 18]
subdata_features = subdata.filter(items=['RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDHHSIZ',
            'INDHHIN2','INDFMIN2', 'BMXBMI', 'URXUAS', 'URXUHG', 'ALQ130', 'DUQ217', 'FSD230', 'INDFMMPI',
            'PAD680', 'OCQ180', 'OCQ210', 'SMQ040', 'SMD650', 'SLD012', 'FIAINTRP', 'FIAPROXY', 'FSD032A', 
            'FSD032B', 'FSD032C', 'FSD061', 'HUQ051', 'HUQ090', 'PAD615','SLQ050','OCQ380', 'PAD630', 'PAD660', 'SLQ120'])

In [10]:
subdata_features.head()

,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,DMDHHSIZ,INDHHIN2,INDFMIN2,BMXBMI,URXUAS,...,FSD032C,FSD061,HUQ051,HUQ090,PAD615,SLQ050,OCQ380,PAD630,PAD660,SLQ120
0,2.0,66.0,4.0,1.0,2.0,1.0,3.0,3.0,31.7,NaN,...,3.0,NaN,2.0,2.0,NaN,2.0,NaN,NaN,NaN,5.397605e-79
1,1.0,18.0,5.0,1.0,NaN,5.0,NaN,NaN,21.5,NaN,...,NaN,NaN,2.0,2.0,NaN,2.0,2.0,NaN,NaN,1.000000e+00
2,2.0,66.0,5.0,1.0,1.0,2.0,6.0,6.0,23.7,24.07,...,3.0,2.0,3.0,2.0,NaN,2.0,4.0,NaN,NaN,2.000000e+00
3,2.0,75.0,4.0,1.0,4.0,1.0,2.0,2.0,38.9,NaN,...,3.0,2.0,3.0,2.0,NaN,2.0,6.0,180.0,NaN,1.000000e+00
4,1.0,56.0,5.0,1.0,5.0,3.0,15.0,15.0,21.3,2.89,...,3.0,NaN,3.0,2.0,NaN,1.0,NaN,NaN,60.0,3.000000e+00


In [11]:
# percentage null values 
nullvals = (subdata.isnull().mean() * 100).round(2)

In [12]:
to_convert = ['RIAGENDR', 'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDHHSIZ', 'INDHHIN2', 'INDFMIN2',
              'FSD230', 'OCQ380', 'OCQ210','SMQ040', 'FIAINTRP', 'FIAPROXY', 'FSD032A', 'FSD061', 'HUQ051', 'HUQ090', 'PAD615', 'PAD630', 'PAD660', 'PAD680', 'SLQ120']


In [13]:
subdata.shape

(5856, 36)

In [14]:
# left join rows with sum_DPQ != null with subdata

valid = valid[features]
valid.columns

sample = pd.merge(valid, subdata, how = "left")
sample.head()

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,DMDHHSIZ,INDHHIN2,INDFMIN2,BMXBMI,...,FSD061,HUQ051,HUQ090,PAD615,SLQ050,OCQ380,PAD630,PAD660,SLQ120,sum_DPQ
0,93705.0,2.0,66.0,4.0,1.0,2.0,1.0,3.0,3.0,31.7,...,NaN,2.0,2.0,NaN,2.0,NaN,NaN,NaN,5.397605e-79,0.0
1,93706.0,1.0,18.0,5.0,1.0,NaN,5.0,NaN,NaN,21.5,...,NaN,2.0,2.0,NaN,2.0,2.0,NaN,NaN,1.000000e+00,0.0
2,93708.0,2.0,66.0,5.0,1.0,1.0,2.0,6.0,6.0,23.7,...,2.0,3.0,2.0,NaN,2.0,4.0,NaN,NaN,2.000000e+00,0.0
3,93711.0,1.0,56.0,5.0,1.0,5.0,3.0,15.0,15.0,21.3,...,NaN,3.0,2.0,NaN,1.0,NaN,NaN,60.0,3.000000e+00,2.0
4,93712.0,1.0,18.0,1.0,2.0,NaN,4.0,4.0,4.0,19.7,...,2.0,1.0,1.0,420.0,2.0,NaN,180.0,45.0,2.000000e+00,1.0


In [15]:
sample.shape

(5068, 37)

## Clean Data

### Original Dataset size: n = 5068


# View Table Before

In [16]:
selected_columns = ['RIAGENDR','RIDRETH1','DMDEDUC2','INDFMIN2','FIAPROXY','FSD032A','HUQ090','SLQ050'
,'SLQ120']
pretty_table = PrettyTable()
pretty_table.add_column("Forward & Backward Selected Variables", selected_columns)
pretty_table.add_column("Percent NA",(sample.loc[:,selected_columns].isnull().mean() * 100).round(1))

cols = full_data.columns.values
values = (full_data.isnull().mean() * 100)
values.index
values = pd.DataFrame({"Var" : values.index.array, "column" :values})

values = values.loc[values["column"]>= 80]

pretty_table = PrettyTable()
pretty_table.add_column("Top 20 Missing Variables", values["Var"][0:20])
pretty_table.add_column("Percent NA", values["column"][0:20].round(1))
pretty_table.sortby = "Percent NA"
pretty_table.reversesort = True
print(pretty_table)

+--------------------------+------------+
| Top 20 Missing Variables | Percent NA |
+--------------------------+------------+
|         BMIHEAD          |   100.0    |
|         BMIRECUM         |    99.7    |
|          BMIHT           |    98.9    |
|         BMXHEAD          |    97.9    |
|          BMIHIP          |    97.1    |
|          BMILEG          |    96.4    |
|         BMIARML          |    96.3    |
|         BMIARMC          |    96.2    |
|          BMIWT           |    95.5    |
|         BMIWAIST         |    95.3    |
|          ALQ290          |    94.4    |
|         DMQADFC          |    93.9    |
|         RIDAGEMN         |    93.5    |
|          DUQ213          |    90.6    |
|         BMXRECUM         |    90.3    |
|         RIDEXPRG         |    88.0    |
|          ALQ280          |    84.4    |
|          ALQ270          |    84.4    |
|          DUQ211          |    81.2    |
|          DUQ210          |    81.2    |
+--------------------------+------

In [17]:
values.size

270

# Filter by threshold

In [18]:

# get columns missing 5% or less values
columns_mostly_complete = sample.columns[(valid.isnull().mean() * 100)  <= 5]


cols_enough_data = ((sample.isnull().mean() * 100) <= 5).array
sample.loc[:,cols_enough_data]
sample = sample.loc[:, columns_mostly_complete.values]




In [19]:
print(sample.shape)
sample.to_csv("out/sample.csv", index=False)

(5068, 22)


# After

## Impute

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# impute using 'median'
imputer = IterativeImputer(missing_values = np.nan, initial_strategy = 'median', random_state= RANDOM_SEED)

imputer = imputer.fit(sample)
filled = imputer.transform(sample)
cols = sample.columns

sample = pd.DataFrame(filled, columns = cols)
sample.shape

(5068, 22)

### Feature selection

In [23]:
(sample.isnull().mean() * 100).head()

SEQN        0.0
RIAGENDR    0.0
RIDAGEYR    0.0
RIDRETH1    0.0
DMDCITZN    0.0
dtype: float64

In [24]:
(sample.isnull().mean() * 100).tail()

HUQ051     0.0
HUQ090     0.0
SLQ050     0.0
SLQ120     0.0
sum_DPQ    0.0
dtype: float64

In [27]:
X = sample.loc[:, ~sample.columns.isin(['SEQN','sum_DPQ'])]
y = sample.loc[:, ['sum_DPQ']]



# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.01, random_state = RANDOM_SEED)


## STANDARDUIZE X_train
normalizer = Normalizer().fit(X_train)
X_scaled = normalizer.transform(X_train)

######################

linear_model = LinearRegression()

sfs1 = SequentialFeatureSelector(linear_model, direction = "backward", cv = 5, n_features_to_select = 8)
sfs1 = sfs1.fit(X_scaled, y_train)

# final selected features
backward_selected_features = X_train.columns.values[sfs1.get_support()]
print(backward_selected_features)

##################

linear_model = LinearRegression()

# forward selection
sfs2 = SequentialFeatureSelector(linear_model, direction = "forward", cv = 5, n_features_to_select = 8)
sfs2 = sfs2.fit(X_scaled, y_train)

# final selected features
forward_selected_features = X_train.columns.values[sfs2.get_support()]
print(forward_selected_features)

['RIAGENDR' 'DMDEDUC2' 'INDFMIN2' 'FIAPROXY' 'FSD032A' 'HUQ090' 'SLQ050'
 'SLQ120']
['RIAGENDR' 'RIDRETH1' 'DMDEDUC2' 'FIAPROXY' 'FSD032A' 'HUQ090' 'SLQ050'
 'SLQ120']


In [26]:
X_train.columns

Index(['RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDHHSIZ',
       'INDHHIN2', 'INDFMIN2', 'BMXBMI', 'PAD680', 'SLD012', 'FIAINTRP',
       'FIAPROXY', 'FSD032A', 'FSD032B', 'FSD032C', 'HUQ051', 'HUQ090',
       'SLQ050', 'SLQ120'],
      dtype='object')

# Fit linear regression

## with all features



In [ ]:
X = sample.loc[:, ~sample.columns.isin(['SEQN','sum_DPQ'])]
y = sample.loc[:, ['sum_DPQ']]

# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.30, random_state = RANDOM_SEED)


# standardize X_train
normalizer = Normalizer().fit(X_train)
X_scaled = normalizer.transform(X_train)


linear_model = LinearRegression()
linear_model = linear_model.fit(X_scaled, y_train)

# predict
y_pred = linear_model.predict(normalizer.transform(X_test))
r2 = r2_score(y_test, y_pred)

MSE = mean_squared_error(y_test, y_pred)
print(f'MSE (All Features): {MSE}\nr-squared: {r2}\nrmse: {np.sqrt(MSE)}')

In [ ]:
plt.scatter(y_test, y_pred, alpha = 0.5)
plt.xlim(0, 30)
plt.ylim(0, 30)
plt.xlabel("true values")
plt.ylabel("pred values")

## with backward selected features

In [ ]:
X = sample.loc[:, sample.columns.isin(backward_selected_features)]
y = sample.loc[:, ['sum_DPQ']]

# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = RANDOM_SEED)


# standardize X_train
normalizer = Normalizer().fit(X_train)
X_scaled = normalizer.transform(X_train)


linear_model = LinearRegression()
linear_model = linear_model.fit(X_scaled, y_train)

# predict
y_pred = linear_model.predict(normalizer.transform(X_test))
r2 = r2_score(y_test, y_pred)

MSE = mean_squared_error(y_test, y_pred)
print(f'MSE (LM Backwards selected features): {MSE}\nr-squared: {r2}\nrmse: {np.sqrt(MSE)}')

In [ ]:
import statsmodels.api as sm
from scipy import stats

## with forward selection

In [ ]:
X = sample.loc[:, sample.columns.isin(forward_selected_features)]
y = sample.loc[:, ['sum_DPQ']]

# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = RANDOM_SEED)


# standardize X_train
normalizer = Normalizer().fit(X_train)
X_scaled = normalizer.transform(X_train)


linear_model = LinearRegression()
linear_model = linear_model.fit(X_scaled, y_train)


# predict
y_pred = linear_model.predict(normalizer.transform(X_test))
r2 = r2_score(y_test, y_pred)

MSE = mean_squared_error(y_test, y_pred)
print(f'MSE (LM Forwards selected features): {MSE}\nr-squared: {r2}\nrmse: {np.sqrt(MSE)}')

In [ ]:
print(sample.loc[sample["sum_DPQ"].between(0,27/2)].shape)
print(sample.loc[sample["sum_DPQ"].between(27/2,28)].shape)

# One hot Encoding sample DF

In [ ]:
import seaborn as sb

corr = sample.loc[:,backward_selected_features].corr()

print(backward_selected_features)
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

# Lasso Regression

In [ ]:

X = sample.loc[:, sample.columns.isin(forward_selected_features)]
y = sample.loc[:, ['sum_DPQ']]

# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = RANDOM_SEED)


clf = LassoCV(n_alphas = 500)
clf = clf.fit(X_train, y_train)

# predict
y_pred = clf.predict(X_test)
r2 = r2_score(y_test, y_pred)

MSE = mean_squared_error(y_test, y_pred)
print(f'MSE (LM Forward selected features): {MSE}\nr-squared: {r2}\nrmse: {np.sqrt(MSE)}')

In [29]:
sample["sum_DPQ"].value_counts()

0.0     1721
1.0      687
2.0      578
3.0      441
4.0      345
5.0      247
6.0      193
7.0      155
8.0      133
9.0      109
10.0      91
11.0      74
12.0      48
14.0      40
13.0      39
16.0      35
15.0      30
18.0      26
17.0      21
21.0      13
19.0      12
20.0      11
22.0      10
23.0       4
24.0       3
25.0       2
Name: sum_DPQ, dtype: int64

## 